In [3]:
%load_ext autoreload

In [4]:
%autoreload 2

In [5]:
import sqlite3

In [6]:
import sys
sys.path.append('../')

# Repos

In [7]:
import requests

base_url = "https://api.github.com"
user = "crawftv"

r = requests.get(f"{base_url}/users/{user}/repos?sort=pushed")


In [8]:
from srv.data_types import RepoResponse
repo = RepoResponse(*list(r.json()[0].values()))

In [66]:
with sqlite3.connect("../portfolio.db") as c:
    c.execute("""
    CREATE TABLE repos 
    (id int PRIMARY KEY, node_id text, name text UNIQUE, full_name text, private int, html_url text, description text, pushed_at_unix real, updated_at_unix real, created_at_unix real)""")

In [67]:
for zz in range(1,6):
    r = requests.get(f"{base_url}/users/{user}/repos?page={zz}")
    for i in r.json():
        repo = RepoResponse(*i.values())
        with sqlite3.connect("portfolio.db") as c:
            repo_ids = [ i[0] for i in c.execute("Select id from repos")]
            try:
                c.execute(f"""INSERT INTO repos VALUES ({repo.id},'{repo.node_id}', "{repo.name}", "{repo.full_name}", {int(repo.private)},"{repo.html_url}","{repo.description}",{repo.pushed_at_unix},{repo.updated_at_unix},{repo.created_at_unix})""")
            except Exception as e:
                print(f"ERROR: {repo.name}, {e}")
        

ERROR: Probabilistic-Programming-and-Bayesian-Methods-for-Hackers, near "Bayesian": syntax error


In [68]:
with sqlite3.connect("../portfolio.db") as c:
    cc = c.cursor().execute(f"""select count(id) from repos""")
    for row in cc:
        print(row)

(90,)


In [65]:
# with sqlite3.connect("portfolio.db") as c:
#     c.cursor().execute(f"""DROP TABLE repos """)

# Events

In [59]:
from srv.data_types import EventResponse
base_url = "https://api.github.com"
user = "crawftv"

In [101]:
# with sqlite3.connect("../portfolio.db") as c:
#     c.execute("""
#     CREATE TABLE events 
#     (id text PRIMARY KEY, type text, created_at_unix real, repo_id int)
#     """)
#     c.execute("""
#     CREATE INDEX repo_event_idx on events(repo_id)""")

In [92]:
for zz in range(1,11):
    r = requests.get(f"{base_url}/users/{user}/events?page={zz}")
    for i in r.json():
        if i["type"] not in ["ForkEvent","WatchEvent"]:
            event = EventResponse(*i.values())
            with sqlite3.connect("portfolio.db") as c:
                c.execute(f"""
                INSERT INTO events
                VALUES ("{event.id}", "{event.type}", {event.created_at_unix},{event.repo_id})
                """)

TypeError: string indices must be integers

In [95]:
r.json()

{'message': "API rate limit exceeded for 108.167.65.65. (But here's the good news: Authenticated requests get a higher rate limit. Check out the documentation for more details.)",
 'documentation_url': 'https://developer.github.com/v3/#rate-limiting'}

In [97]:
# with sqlite3.connect("portfolio.db") as c:
#     c.cursor().execute(f"""DROP TABLE events """)